## DASH APP

In [12]:

# --- Lancer ---
if __name__ == "__main__":
    import webbrowser
    url = "http://127.0.0.1:8050/"
    webbrowser.open_new_tab(url)
    app.run(debug=True, port=8050, threaded=True)


In [11]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import json
import numpy as np
from pathlib import Path
from datetime import datetime
import sys
from pathlib import Path
import importlib
import ipynbname 
import pandas as pd
import geopandas as gpd
from datetime import datetime

code_path = ipynbname.path().parent.parent
# Ajouter le dossier scripts au path
scripts_path = code_path  / "scripts"
base_path=code_path.parent
sys.path.append(str(scripts_path.resolve()))

import data_utils  # importe le module une première fois

# Après avoir modifié data_utils.py
importlib.reload(data_utils)

# Maintenant tu peux accéder aux fonctions mises à jour
from data_utils import import_data_raw, import_data_sig, melt_long_format, clean_year_column, save_long_dataframe, concat_intermediate_files

# --- Fonctions utilitaires ---
def symlog(x):
    """Logarithme symétrique : log10(|x|+1) * signe(x)"""
    if pd.isna(x):
        return None
    return np.sign(x) * np.log10(abs(x) + 1)

# --- Charger données ---
# Remplacer ces chemins par vos fichiers
filename="data_final_all_norm.csv"
filepath= base_path/ "Data" / 'data_final' / filename

df_data = pd.read_csv(filepath)

#Global sig
gdf_world=import_data_sig('world.geojson',base_path)

# --- Normalisations et colonnes d'unité ---
norm_map = {
    'No norm': ('Value', 'Unit'),
    'Area': ('Value_norm_area', 'Unit_norm_area'),
    'Population': ('Value_norm_population', 'Unit_norm_population'),
    'Hab/km2': ('Value_norm_densite', 'Unit_norm_densite'),
    'PPP': ('Value_norm_ppp', 'Unit_norm_ppp'),
    'GDP': ('Value_norm_gdp', 'Unit_norm_gdp'),
    'PPP/hab': ('Value_norm_ppp_hab', 'Unit_norm_ppp_hab'),
    'GDP/hab': ('Value_norm_gdp_hab', 'Unit_norm_gdp_hab')
}

def simplify_geom(geom, tol=0.1):
    if geom is None:
        return None
    if geom.geom_type == 'Polygon':
        return geom.simplify(tol, preserve_topology=True)
    elif geom.geom_type == 'MultiPolygon':
        return type(geom)([poly.simplify(tol, preserve_topology=True) for poly in geom.geoms])
    return geom
    
gdf_world = gdf_world[gdf_world['Country_code'].notna()].copy()
gdf_world['geometry'] = gdf_world['geometry'].apply(lambda g: simplify_geom(g, tol=0.1))
# --- Dash app ---
app = dash.Dash(__name__)
@app.callback(
    Output("world_map", "figure"),
    Input("indicator", "value"),
    Input("database", "value"),
    Input("year", "value"),
    Input("scale", "value"),
    Input("color_range", "value"),
    Input("normalization", "value")  # <- add normalization dropdown
)
def update_map(indicator, database, year, scale, color_range, normalization):

    df_filtered = df_data[(df_data['Indicator'] == indicator) & 
                          (df_data['Source'] == database) & 
                          (df_data['Year'] == year)].copy()
    gdf_merged = gdf_world.merge(df_filtered, on='Country_code', how='left')
    geojson_data = json.loads(gdf_merged.to_json())



    col_value, col_unit = norm_map[normalization]

    if col_value not in gdf_merged.columns:
        gdf_merged[col_value] = np.nan
    if col_unit not in gdf_merged.columns:
        gdf_merged[col_unit] = ""

    z_values = gdf_merged[col_value]
    
    # --- Compute z range ---
    if color_range == 'raw':
        zmin, zmax = z_values.min(), z_values.max()
    elif color_range.startswith("q"):
        q_low = float(color_range[1:])
        q_high = 1 - q_low
        zmin, zmax = z_values.quantile(q_low), z_values.quantile(q_high)
    elif color_range.startswith("*"):
        factor = float(color_range[1:])
        zmin, zmax = factor * z_values.min(), factor * z_values.max()
    else:
        raise ValueError("color_range must be 'raw', 'q0.xx', or '*0.xx'")

    # --- Apply scale ---
    if scale == 'rank':     
        z_plot = z_values.rank(ascending=True)
        zmin, zmax = z_plot.min(), z_plot.max()  # <-- important
    elif scale == 'absolute':
        z_plot = z_values.copy()
        zborne = max(abs(zmin), abs(zmax))
        zmin, zmax = -zborne, zborne
    elif scale == 'relative':
        z_plot = z_values.copy()
    elif scale == 'log':
        z_plot = z_values.apply(symlog)
        zmin, zmax = None, None
    else:
        raise ValueError("scale must be 'absolute', 'relative', 'rank', or 'log'")

    # --- Colorbar ticks ---
    if scale in ['absolute', 'relative']:
        N_ticks = 8
        tickvals = np.linspace(zmin, zmax, N_ticks)
        ticktext = [str(int(v)) if abs(zmax-zmin)>10 else str(round(v,2)) for v in tickvals]
    else:
        tickvals = None
        ticktext = None

    # --- Choropleth ---
    fig = go.Figure(go.Choropleth(
        geojson=geojson_data,
        locations=gdf_merged.index,
        z=z_plot,
        text=gdf_merged['name'],
        colorscale='RdYlGn_r',
        zmin=zmin,
        zmax=zmax,
        zmid=0 if scale=='absolute' else None,
        customdata=z_values,
        colorbar=dict(
            title="Rank" if scale == "rank" else (
                gdf_merged[col_unit].iloc[0] if col_unit in gdf_merged.columns else ""
            ),
            tickvals=tickvals,
            ticktext=ticktext,
            x=0.02, xanchor='left'
        ),
        hovertemplate=f"<b>%{{text}}</b><br>{indicator} ({normalization}) = %{{customdata:.2f}} {gdf_merged[col_unit].iloc[0]}<extra></extra>"
    ))

    fig.update_layout(
        geo=dict(
            scope="world", projection_type="natural earth",
            showcountries=True, showcoastlines=True, showland=True, showocean=True,
            landcolor="lightgray", oceancolor="lightblue",lakecolor="lightblue"
        ),
        #height=200
    )

    return fig

# --- Options pour les menus ---
indicator_options = df_data['Indicator'].unique()
database_options = df_data['Source'].unique()
year_options = df_data['Year'].unique()
scale_options = ['absolute', 'relative', 'rank', 'log']
color_range_options = ['raw', 'q0.01', 'q0.05', 'q0.1', '*0.8', '*1.2']

# --- Sidebar options for normalization ---
norm_map_labels = list(norm_map.keys())  # ["No norm", "Area", "Population", ...]

# --- Dash app layout ---
app.layout = html.Div([
    html.Div([
        # --- Sidebar for controls ---
        html.Div([
            html.Div([
                html.Label("Indicator"),
                dcc.Dropdown(
                    id="indicator",
                    options=[{"label": i, "value": i} for i in indicator_options],
                    value=indicator_options[0],
                    placeholder="Indicator",
                    style={'marginBottom': '20px'}
                )
            ]),

            html.Div([
                html.Label("Database"),
                dcc.Dropdown(
                    id="database",
                    options=[{"label": i, "value": i} for i in database_options],
                    value=database_options[0],
                    placeholder="Database",
                    style={'marginBottom': '20px'}
                )
            ]),

            html.Div([
                html.Label("Year"),
                dcc.Dropdown(
                    id="year",
                    options=[{"label": str(y), "value": y} for y in sorted(year_options)],
                    value=min(year_options),
                    placeholder="Year",
                    style={'marginBottom': '20px'}
                )
            ]),

            html.Div([
                html.Label("Normalization"),
                dcc.Dropdown(
                    id="normalization",
                    options=[{"label": name, "value": name} for name in norm_map_labels],
                    value=norm_map_labels[0],
                    placeholder="Normalization",
                    style={'marginBottom': '20px'}
                )
            ]),

            html.Div([
                html.Label("Scale"),
                dcc.Dropdown(
                    id="scale",
                    options=[{"label": i, "value": i} for i in scale_options],
                    value='relative',
                    placeholder="Scale",
                    style={'marginBottom': '20px'}
                )
            ]),

            html.Div([
                html.Label("Color Range"),
                dcc.Dropdown(
                    id="color_range",
                    options=[{"label": i, "value": i} for i in color_range_options],
                    value='raw',
                    placeholder="Color range",
                    style={'marginBottom': '20px'}
                )
            ]),

        ], style={
            'flex': '0 0 250px',
            'padding': '15px',
            'backgroundColor': '#f8f9fa',
            'boxShadow': '2px 0px 5px rgba(0,0,0,0.1)',
            'height': '90vh',
            'overflowY': 'auto'
        }),

        # --- Main content: map ---
        html.Div([
            dcc.Graph(id="world_map", style={'height': '100%', 'width': '100%'})
        ], style={'flex': '1', 'padding': '10px', 'minHeight': '800px'}),

    ], style={'display': 'flex', 'flexDirection': 'row', 'height': '90vh'}),
])





### V2